# Analyzing borrowers’ risk of defaulting

Your project is to prepare a report for a bank’s loan division. You’ll need to find out if a customer’s marital status and number of children has an impact on whether they will default on a loan. The bank already has some data on customers’ credit worthiness. Your report will be considered when building a **credit scoring** of a potential customer.

## Open the data file and have a look at the general information. 

In [1]:
import pandas as pd
import nltk
from nltk.stem import WordNetLemmatizer
from collections import Counter
import warnings

warnings.filterwarnings('ignore')

try:
    df = pd.read_csv('credit_scoring_eng.csv')
    nltk.download('punkt')
    nltk.download('wordnet')
except:
    df = pd.read_csv('/datasets/credit_scoring_eng.csv')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/jasonreaves/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/jasonreaves/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


### info()

In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


### head() & tail()

In [3]:
df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house
1,1,-4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase
2,0,-5623.422610,33,Secondary Education,1,married,0,M,employee,0,23341.752,purchase of the house
3,3,-4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education
4,0,340266.072047,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding


In [4]:
df.tail()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
21520,1,-4529.316663,43,secondary education,1,civil partnership,1,F,business,0,35966.698,housing transactions
21521,0,343937.404131,67,secondary education,1,married,0,F,retiree,0,24959.969,purchase of a car
21522,1,-2113.346888,38,secondary education,1,civil partnership,1,M,employee,1,14347.610,property
21523,3,-3112.481705,38,secondary education,1,married,0,M,employee,1,39054.888,buying my own car
21524,2,-1984.507589,40,secondary education,1,married,0,F,employee,0,13127.587,to buy a car


### Conclusion

| Column | info() | head() & tail() |
|---|---|---|
| children | OK | OK |
| days_employed | 2,174 missing values<br>Change to int | Negative numbers for most<br>Huge numbers for retirees
| dob_years | OK | OK |
| education | OK | Capitalization |
| education_id | OK | OK |
| family_status | OK | OK |
| family_status_id | OK | OK |
| gender | OK | OK |
| income_type | OK | OK |
| debt | OK | OK |
| total_income | 2,174 missing values | OK |
| purpose | OK | Inconsistent wording |

- There are 21,525 entries in the DataFrame. Most columns have all 21,525 entries, but <code>days_employed</code> and <code>total_income</code> appear to have 2,174 missing values each.
- In addition to having missing values, <code>days_employed</code> has spurious data, including negative values and impossibly large values.
- The <code>education</code> field has inconsistent capitalization.
- The <code>purpose</code> field has in inconsistent wording. Different words are used to describe the same thing, possibly because a user entered the value as free text. This field will need to be standardized.
- At first glance, the other fields look okay, but we will need to do a "reality check" on each one.

## Data preprocessing

### Processing missing values
In this section, we will be processing missing values in the <code>days_employed</code> and <code>total_income</code> columns.  However, our preliminary analysis uncovered spurious values for <code>days_employed</code>.  This must be addressed before missing values can be handled.

#### Spurious values in the <code>days_employed</code> column
Browsing through the data reveals that there are two types of values in the <code>days_employed</code> column: negative values and impossibly large ones. We can confirm that none of the values fall in a reasonable range for days employed, e.g., 0 through 29,200 days (which is 80 years).

In [5]:
df[(df['days_employed'] < 29200) & (df['days_employed'] >= 0)]['days_employed'].count()

0

Let's focus on the huge values first.

In [6]:
df[df['days_employed'] > 29200].head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
4,0,340266.072047,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding
18,0,400281.136913,53,secondary education,1,widow / widower,2,F,retiree,0,9091.804,buying a second-hand car
24,1,338551.952911,57,secondary education,1,unmarried,4,F,retiree,0,46487.558,transactions with commercial real estate
25,0,363548.489348,67,secondary education,1,married,0,M,retiree,0,8818.041,buy real estate
30,1,335581.668515,62,secondary education,1,married,0,F,retiree,0,27432.971,transactions with commercial real estate


These appear to be retirees.  Let's see if any _aren't_ retirees.

In [7]:
df[(df['days_employed'] > 29200) & (df['income_type'] != 'retiree')]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
3133,1,337524.466835,31,secondary education,1,married,0,M,unemployed,1,9593.119,buying property for renting out
14798,0,395302.838654,45,Bachelor's Degree,0,civil partnership,1,F,unemployed,0,32435.602,housing renovation


All but two are retirees, and they are both unemployed.  Since the records with high values all represent retirees or unemployed, we will set <code>days_employed</code> for these rows to zero.

In [8]:
# We showed earlier that > 29200 catches all of the high values.
df.loc[df['days_employed'] > 29200, 'days_employed'] = 0
df.loc[df['days_employed'] > 29200, 'days_employed']

Series([], Name: days_employed, dtype: float64)

What about the negative values? They represent the entire data set for <code>days_employed</code>, with the exception of the huge values processed above.  I suspect that the negative sign came from reasoning like, "From today, when did the loan applicant start their current job?" The answer, in that case, would be a negative number.  Another explanation would be a dash instead of a minus.  In order words, the data is valid, except for the negative sign in front.  I'll address this by replacing each value with its absolute value.

In [9]:
df['days_employed'] = abs(df['days_employed'])

As a final reality check, let's check the range for <code>days_employed</code>.

In [10]:
min_days_employed = int(df['days_employed'].min())
max_days_employed = int(df['days_employed'].max())
max_years_employed = int(max_days_employed/365)
print("days_employed ranges from {} to {}, which is {} years."
      .format(min_days_employed, max_days_employed, max_years_employed)
)

days_employed ranges from 0 to 18388, which is 50 years.


#### Missing values in <code>days_employed</code> and <code>total_income</code> columns

As noted above, <code>days_employed</code> and <code>total_income</code> each have 2,174 missing values.  Let's see what percentage of the data set this represents:

In [11]:
pd.DataFrame(df.isna().mean().round(4)*100).style.background_gradient('coolwarm')

,0
children,0.000000
days_employed,10.100000
dob_years,0.000000
education,0.000000
education_id,0.000000
family_status,0.000000
family_status_id,0.000000
gender,0.000000
income_type,0.000000
debt,0.000000


In [12]:
df[df['days_employed'].isnull()].head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,secondary education,1,civil partnership,1,M,retiree,0,NaN,to have a wedding
26,0,NaN,41,secondary education,1,married,0,M,civil servant,0,NaN,education
29,0,NaN,63,secondary education,1,unmarried,4,F,retiree,0,NaN,building a real estate
41,0,NaN,50,secondary education,1,married,0,F,civil servant,0,NaN,second-hand car purchase
55,0,NaN,54,secondary education,1,civil partnership,1,F,retiree,1,NaN,to have a wedding


It appears that rows with missing values for <code>days_employed</code> also have missing values for <code>total_income</code>. We can confirm that these are all the same rows as follows:

In [13]:
days_employed_indices = df[df['days_employed'].isnull()].index.tolist()
total_income_indices = df[df['total_income'].isnull()].index.tolist()

if set(days_employed_indices) == set(total_income_indices):
    print('Missing values for days_employed and missing values for total_income share the same rows.')

Missing values for days_employed and missing values for total_income share the same rows.


Note that these people are not all retirees or unemployed. Also, it is possible for reitrees and unemployed people to have non-employment income. For these reasons, we will choose non-zero default values.  For <code>days_cmployed</code>, we will fill in the mean value.

In [14]:
df.loc[days_employed_indices, 'days_employed'] = df['days_employed'].mean(skipna=True)

For <code>total_income</code>, we will fill in the median value based on the applicant's education level and income type.  First, we create a lookup table with the median incomes for each <code>education</code> and <code>income_type</code> pair:

In [15]:
# Note: The following line fixes the duplicates in the education column
df['education'] = df['education'].str.lower()

education = df['education_id'].unique()
columns = ['education']
income_types = df['income_type'].unique()
columns.extend(income_types)
income_table = pd.DataFrame(columns=columns)

for e in education:
    income_table.loc[e, 'education'] = df[df['education_id'] == e]['education'].iloc[0]
    for t in income_types:
        income_table.loc[e, t] = df[(df['education_id'] == e) & (df['income_type'] == t)]['total_income'].median(skipna=True)

income_table.fillna(0)

,education,employee,retiree,business,civil servant,unemployed,entrepreneur,student,paternity / maternity leave
0,bachelor's degree,26502.5190,23078.523,32285.664,27601.7775,32435.602,79866.103,15712.26,0.000
1,secondary education,21848.8175,18374.857,25451.310,21864.4750,9593.119,0.000,0.00,8612.661
2,some college,24209.4300,19221.903,28778.744,25694.7750,0.000,0.000,0.00,0.000
3,primary education,20159.1860,16415.785,21887.825,23734.2870,0.000,0.000,0.00,0.000
4,graduate degree,31771.3210,28334.215,0.000,17822.7570,0.000,0.000,0.00,0.000


Next, we will use this lookup table to supply default values for the missing values in the <code>total_income</code> column.

In [16]:
def add_default_income(row):
    return income_table.loc[row['education_id'], row['income_type']]

df.loc[total_income_indices, 'total_income'] = df.loc[total_income_indices].apply(add_default_income, axis=1)

df.loc[total_income_indices].tail()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
21489,2,1934.115623,47,secondary education,1,married,0,M,business,0,25451.3100,purchase of a car
21495,1,1934.115623,50,secondary education,1,civil partnership,1,F,employee,0,21848.8175,wedding ceremony
21497,0,1934.115623,48,bachelor's degree,0,married,0,F,business,0,32285.6640,building a property
21502,1,1934.115623,42,secondary education,1,married,0,F,employee,0,21848.8175,building a real estate
21510,2,1934.115623,28,secondary education,1,married,0,F,employee,0,21848.8175,car purchase


Missing values for <code>days_employed</code> have been replaced with the mean value for that column.  Missing values for <code>total_income</code> have been replaced with median values, taking education level and income type into account.

### Conclusion
In this section, we began by filling in the spurious values for <code>days_employed</code>.  Impossibly high values represented retirees (and a couple of unemployed), so these were set to zero.  The negative values seemed to be valid other than their sign and were replaced with their absolute value.  The range of <code>days_employed</code> was then checked and was reasonable.

We investigated the missing values in the <code>days_employed</code> and <code>total_income</code> columns. We found that missing values for the columns shared the same rows but did not exhibit an obvious pattern in terms of other columns.  These "missing at random" values were replaced with mean and median default values.  The mean value was used for <code>days_employed</code>.  Median values, taking into account the applicant's education level and employment type, were used for <code>total_income</code>.

How did the missing data arrive in the first place? <code>days_employed</code> and <code>total_income</code> were missing in exactly the same rows.  This suggests that the user interface of the software used by the borrower or loan officer links the two fields.  For instance, if the user fails to enter the employment length, the income field may not appear.

### Processing spurious values
In our initial look at the data, some columns looked "OK" but bad data could easily be hiding outside of the first and last 5 rows.  In this section, we examine each column in further detail, looking for any data that doesn't make sense.

#### children

In [17]:
df['children'].value_counts().sort_values(ascending=False)

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

Right off the bat, we have 47 rows with negative children, and 76 rows with 20 children.  Both of these are obviously erroneous.  (It is conceivable that someone could have 20 children, but it is inconveivable that the same data set would have no one with 19, 18...6 children, and that there would be more people with 20 children than with 5 children.)  First, let's look at the rows with negative children.

In [18]:
df[df['children'] == -1].head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
291,-1,4417.703588,46,secondary education,1,civil partnership,1,F,employee,0,16450.615,profile education
705,-1,902.084528,50,secondary education,1,married,0,F,civil servant,0,22061.264,car purchase
742,-1,3174.456205,57,secondary education,1,married,0,F,employee,0,10282.887,supplementary education
800,-1,0.000000,54,secondary education,1,unmarried,4,F,retiree,0,13806.996,supplementary education
941,-1,1934.115623,57,secondary education,1,married,0,F,retiree,0,18374.857,buying my own car


I see no obvious pattern explaining why these rows have negative children.  Since the error appears to be random, a default value will be assigned.  Two approaches are evident:

1. Fill all with 0, since thats the most frequent number of children (i.e., the mode).
2. Fill in with different numbers of children based on a breakdown of number of children by something like age or marital status.

The second option seems like a lot of work for little benefit. (The negatives only represent about 0.2% of the data.)  So, we will fill in zeros.

In [19]:
df.loc[df['children'] == -1, 'children'] = 0

Next we turn our attention to the weird rows with 20 children.

In [20]:
df[df['children'] == 20].head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
606,20,880.221113,21,secondary education,1,married,0,M,business,0,23253.578,purchase of the house
720,20,855.595512,44,secondary education,1,married,0,F,business,0,18079.798,buy real estate
1074,20,3310.411598,56,secondary education,1,married,0,F,employee,1,36722.966,getting an education
2510,20,2714.161249,59,bachelor's degree,0,widow / widower,2,F,employee,0,42315.974,transactions with commercial real estate
2941,20,2161.591519,0,secondary education,1,married,0,F,employee,0,31958.391,to buy a car


Again, no pattern emerges, and again, we will replace the values with zeros.

In [21]:
df.loc[df['children'] == 20, 'children'] = 0
df['children'].value_counts().sort_values(ascending=False)

0    14272
1     4818
2     2055
3      330
4       41
5        9
Name: children, dtype: int64

#### days_employed
See previous section.

#### dob_years

In [22]:
df['dob_years'].value_counts().sort_index()

0     101
19     14
20     51
21    111
22    183
23    254
24    264
25    357
26    408
27    493
28    503
29    545
30    540
31    560
32    510
33    581
34    603
35    617
36    555
37    537
38    598
39    573
40    609
41    607
42    597
43    513
44    547
45    497
46    475
47    480
48    538
49    508
50    514
51    448
52    484
53    459
54    479
55    443
56    487
57    460
58    461
59    444
60    377
61    355
62    352
63    269
64    265
65    194
66    183
67    167
68     99
69     85
70     65
71     58
72     33
73      8
74      6
75      1
Name: dob_years, dtype: int64

We have 101 zero-year-olds. Replace with the mean age, based on a new column, <code>years_employed</code>.

In [23]:
df['years_employed'] = (df['days_employed']/365).astype('int')

def guess_dob(row):
    return int(df[df['years_employed'] == row['years_employed']]['dob_years'].mean(skipna=True))

df.loc[df['dob_years'] == 0, 'dob_years'] = df.apply(guess_dob, axis=1)
df['dob_years'].value_counts().sort_index()

19     14
20     51
21    111
22    183
23    254
24    264
25    357
26    408
27    493
28    503
29    545
30    540
31    560
32    510
33    581
34    603
35    617
36    569
37    549
38    610
39    577
40    611
41    611
42    615
43    518
44    547
45    497
46    477
47    481
48    538
49    510
50    514
51    473
52    484
53    459
54    479
55    443
56    487
57    460
58    461
59    444
60    377
61    355
62    352
63    269
64    265
65    194
66    183
67    167
68     99
69     85
70     65
71     58
72     33
73      8
74      6
75      1
Name: dob_years, dtype: int64

#### education
See below (Processing Duplicates).

#### family_status

In [24]:
df['family_status'].value_counts().sort_index()

civil partnership     4177
divorced              1195
married              12380
unmarried             2813
widow / widower        960
Name: family_status, dtype: int64

This one looks okay.

#### gender

In [25]:
df['gender'].value_counts().sort_index()

F      14236
M       7288
XNA        1
Name: gender, dtype: int64

In [26]:
df[df['gender'] == 'XNA']

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,years_employed
10701,0,2358.600502,24,some college,2,civil partnership,1,XNA,business,0,32624.825,buy real estate,6


They're in a civil partnership, so I suspect this is not a typo. No change.

#### debt

In [27]:
df['debt'].value_counts()

0    19784
1     1741
Name: debt, dtype: int64

Looks correct.

#### total_income

In [28]:
min_income = df['total_income'].min()
max_income = df['total_income'].max()
print("Min income: {}\nMax income: {}\n".format(min_income, max_income))

Min income: 3306.762
Max income: 362496.645



Again, seems reasonable.

#### purpose

The purpose field will be processed to standardize its values. This will be covered below, in the Categorizing Data section.

### Conclusion
In this section, we examined each column in more detail.  While some of the columns seemed correct, we found and corrected a couple of errors:
- Families with negative children and 20 children became childless.
- Zero-year-olds were assigned the average age, based on how many years they've been employed.

### Data type replacement

#### days_employed
Overall, the data types in the initial data set seem appropriate.  One possible exception is <code>days_employed</code>, which is a float but should probably be an integer.

In [29]:
df['days_employed'] = df['days_employed'].astype('int')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     21525 non-null  int64  
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      21525 non-null  float64
 11  purpose           21525 non-null  object 
 12  years_employed    21525 non-null  int64  
dtypes: float64(1), int64(7), object(5)
memory usage: 2.1+ MB


### Conclusion
<code>astype()</code> was used to change <code>days_employed</code> from float to int.
- <code>astype()</code> is the preferred approach when the column you want to convert is already a numeric type.
- <code>to_numeric()</code> can be used when some of the data is not already numeric type.  It allows you to specify how to handle non-numeric input.

### Processing duplicates
Our initial look at the data revealed that the <code>education</code> field had inconsistent capitalization.   Fortunately, there were no other inconsistencies, such as punctuation or spelling errors. We already fixed the capitalization in the Processing missing values section above, with the following code:

``` python
df['education'] = df['education'].str.lower()
```

In [30]:
df['education'].value_counts()

secondary education    15233
bachelor's degree       5260
some college             744
primary education        282
graduate degree            6
Name: education, dtype: int64

### Conclusion
The duplicate data (inconsistent capitalization) in the <code>education</code> column) may have resulted from manual data entry.  However, I would actually expect greater inconsistency in the data if it were entered by the borrowers themselves.  Another possible explanation for the the observed pattern of inconsistency could be changes in the software used to input or collect the data.  This could produce differences in capitalization without spelling or punctuation errors.

### Categorizing Data

#### Standardizing the purpose column
Our preliminary look at the data showed inconsistencies in the <code>purpose</code> column.  In particular, the same purpose was described in many different ways, and some descriptions were ambiguous.  The goal is to replace the current hodgepodge of purposes with a smaller, standardized list.  The overall approach is as follows:

1. Lemmatize the purpose column.
2. Examine the list of lemmas for patterns.
3. Find a small, yet descriptive, set of purposes.
4. Apply the standardized purposes to the DataFrame.

##### Lemmatize the purpose column
First, we gather all of the lemmas in the data for analysis.  We use Counter to pick out the unique ones.

In [31]:
purposes = ' '.join(df['purpose'].unique())
words = nltk.word_tokenize(purposes)
wordnet_lemma = WordNetLemmatizer()
lemmas = Counter([wordnet_lemma.lemmatize(w, pos='n') for w in words])
lemmas

Counter({'purchase': 6,
         'of': 5,
         'the': 2,
         'house': 3,
         'car': 9,
         'supplementary': 2,
         'education': 7,
         'to': 6,
         'have': 1,
         'a': 9,
         'wedding': 3,
         'housing': 3,
         'transaction': 4,
         'having': 1,
         'for': 2,
         'my': 4,
         'family': 1,
         'buy': 4,
         'real': 7,
         'estate': 7,
         'commercial': 2,
         'residential': 1,
         'construction': 1,
         'own': 4,
         'property': 4,
         'building': 2,
         'buying': 3,
         'second-hand': 2,
         'with': 2,
         'become': 1,
         'educated': 1,
         'getting': 2,
         'an': 1,
         'ceremony': 1,
         'get': 1,
         'higher': 1,
         'profile': 1,
         'university': 2,
         'renting': 1,
         'out': 1,
         'renovation': 1,
         'going': 1})

##### Examine the list of lemmas for patterns

Examining the lemmas, there appear to be three categories:

1. Lemmas that clearly imply a particular purpose. These will become part of our standadized purpose.
    - 'house', 'car', 'education', 'wedding', 'housing'
2. Lemmas that carry no useful meaning. These will be ignored.
    - 'purchase', 'purpose', 'of', 'the', 'to', 'have', 'a', 'having', 'for', 'my', 'buy', 'own', 'buying', 'second-hand', 'with', 'become', 'getting', 'an', 'get', 'out', 'going'
3. Lemmas that require further investigation. These will be flagged for further analysis.

For step 3, we will create a table relating the flagged lemmas and to how they appear in the <code>purpose</code> column.

In [32]:
# These require further investigation
flagged = ['supplementary', 'transaction', 'family', 'real', 'estate', 'commercial', 'residential', 'construction', 'property', 'building', 'educated', 'ceremony', 'higher', 'profile', 'university', 'renting', 'renovation']

wordnet_lemma = WordNetLemmatizer()

def get_lemmas(purpose):
    words = nltk.word_tokenize(purpose)    
    lemmas = [wordnet_lemma.lemmatize(w, pos='n') for w in words]
    return lemmas

# Create columns with lemmatized versions of purpose
df['purpose_lemmas'] = df['purpose'].apply(get_lemmas)
df['purpose_lemmas_str'] = df.apply(lambda r: ' '.join(r['purpose_lemmas']), axis=1)

flag_data = pd.DataFrame(columns=['flag','purpose','count'])

for f in flagged:
    purposes = df[df['purpose_lemmas_str'].str.contains(f)]['purpose_lemmas_str'].unique()
    for p in purposes:    
        count = len(df[df['purpose_lemmas_str'] == p])
        row = pd.DataFrame([[f, p, count]], columns=['flag','purpose','count'])
        flag_data = pd.concat([flag_data, row], ignore_index=True)

flag_data

,flag,purpose,count
0,supplementary,supplementary education,462
1,supplementary,to get a supplementary education,447
2,transaction,housing transaction,653
3,transaction,transaction with commercial real estate,651
4,transaction,transaction with my real estate,630
5,transaction,real estate transaction,676
6,family,purchase of the house for my family,641
7,real,buy real estate,624
8,real,buy commercial real estate,664
9,real,buy residential real estate,607


Reviewing how the flagged lemmas appear in the original purpose field, several patterns emerge.

1. Some of the flagged lemmas are redundant.  For instance, "supplementary" always appears with "education".  Since we've already decided to use "education", "supplementary" can be ignored.
    - Other lemmas that can be ignored include: family, residential, ceremony, higher, profile, university
2. "real" always occurs with "estate".  One of them can be safely ignored.
3. "real estate" is ambiguous.  It can refer to specific purposes such as residential and commercial, but also to vague purposes such as "real estate transaction".  I have elected to treat residential as "house" and commercial as "commercial real estate".  More vague uses will be assigned to "real estate other" if a more specific purposes is not available.
    - Similar lemmas: transaction, property
4. We will keep "building" and "construction" to refer to "new construction".
5. "renting" will be kept, as "buy to rent"
6. "renovation" will be kept as "home renovation". Note that we have to make sure it is treated with higher priority that "housing", which would otherwise be assigned to "house".

In the next section, these ideas are implemented using a dictionary to map lemmas to their standard purpose description. (Scroll the output to the right and behold the new <code>std_purpose</code> column!)

In [33]:
# From python 3.6 onwards, dict maintains keys in insertion order.
# We're relying on this: "renovation" must come before "housing"
# so that "housing renovation" is classified correctly:  only if
# there's no match on "renovation" is "housing" converted to "house".
# The same goes for other generic classifiers like "property",
# "transaction", and "real estate". More specific classifications are
# preferred over generic ones like "real estate other" and "property other".
purpose_dict = {
                "house":"house",
                "car":"car",
                "education":"education",
                "wedding": "wedding",
                "commercial":"commercial real estate",
                "residential":"house",
                "construction":"new construction",
                "building":"new construction",
                "educated":"education",
                "university":"education",
                "renting":"buy to rent",
                "renovation":"home renovation",
                "housing":"house",
                "transaction":"real estate other",
                "real":"real estate other",
                "property":"property other"
}

purpose_dict_keys = list(purpose_dict.keys())

# Search for all purchase_lemmas in purpose_dict. Return the
# match with the lowest index in purpose_dict. The match with
# the lowest index is the most specific one.
def standardize_purpose(row):
    purpose_lemmas = row['purpose_lemmas']
    purpose_std = ''
    match = ''
    min_match_index = None
    for p in purpose_lemmas:
        match = purpose_dict.get(p)
        if match != None:
            # Got a match
            match_index = purpose_dict_keys.index(p)
            if min_match_index == None: # First match
                purpose_std = match
                min_match_index = match_index
            else:
                if match_index < min_match_index:
                    # Replace previous match with current
                    # one only if current one has lower index.
                    purpose_std = match
                    min_match_index = match_index
    if purpose_std != '':
        return purpose_std
    else:
        # With the dictionary complete, we shouldn't get here.
        print(purpose_lemmas)
        return None

df['purpose_std'] = df.apply(standardize_purpose, axis=1)
df.tail(15)
# Note that "buying property for renting out" was assigned correctly. Even
# though "property" matched first, "buy to rent" was correctly assigned
# because it had a lower index in purpose_dict. Similarly, "home renovation"
# we maintained as higher priority than "housing".

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,years_employed,purpose_lemmas,purpose_lemmas_str,purpose_std
21510,2,1934,28,secondary education,1,married,0,F,employee,0,21848.8175,car purchase,5,"[car, purchase]",car purchase,car
21511,0,612,29,bachelor's degree,0,civil partnership,1,F,employee,1,22410.9560,buying property for renting out,1,"[buying, property, for, renting, out]",buying property for renting out,buy to rent
21512,0,165,26,bachelor's degree,0,unmarried,4,M,business,0,23568.2330,to get a supplementary education,0,"[to, get, a, supplementary, education]",to get a supplementary education,education
21513,0,1166,35,secondary education,1,married,0,F,employee,0,40157.7830,purchase of the house,3,"[purchase, of, the, house]",purchase of the house,house
21514,0,280,27,some college,2,unmarried,4,M,business,0,56958.1450,building a property,0,"[building, a, property]",building a property,new construction
21515,1,467,28,secondary education,1,married,0,F,employee,1,17517.8120,to become educated,1,"[to, become, educated]",to become educated,education
21516,0,914,42,bachelor's degree,0,married,0,F,business,0,51649.2440,purchase of my own house,2,"[purchase, of, my, own, house]",purchase of my own house,house
21517,0,404,42,bachelor's degree,0,civil partnership,1,F,business,0,28489.5290,buying my own car,1,"[buying, my, own, car]",buying my own car,car
21518,0,0,59,secondary education,1,married,0,F,retiree,0,24618.3440,purchase of a car,0,"[purchase, of, a, car]",purchase of a car,car
21519,1,2351,37,graduate degree,4,divorced,3,M,employee,0,18551.8460,buy commercial real estate,6,"[buy, commercial, real, estate]",buy commercial real estate,commercial real estate


#### Adding dictionaries for the <code>income_type</code> and <code>purpose_std</code> columns
In this section we will add a <code>id</code> columns for <code>income_type</code> and <code>purpose_std</code>.  This enables the creation of dictionaries linking the id to the column text.

##### <code>income_type</code>

In [34]:
# Get a sorted list of income types
income_types = df['income_type'].value_counts().index

# Assign id's to each income type
for i, t in enumerate(income_types):
    df.loc[df['income_type'] == t, 'income_type_id'] = i

df['income_type_id'] = df['income_type_id'].astype('int')

# Print the id's along with their description
df[['income_type_id', 'income_type']].value_counts()

income_type_id  income_type                
0               employee                       11119
1               business                        5085
2               retiree                         3856
3               civil servant                   1459
4               unemployed                         2
5               entrepreneur                       2
6               student                            1
7               paternity / maternity leave        1
dtype: int64

##### <code>purpose_std</code>

In [35]:
purposes = df['purpose_std'].value_counts().index

for i, p in enumerate(purposes):
    df.loc[df['purpose_std'] == p, 'purpose_std_id'] = i

df['purpose_std_id'] = df['purpose_std_id'].astype('int')

df[['purpose_std_id', 'purpose_std']].value_counts()

purpose_std_id  purpose_std           
0               car                       4315
1               education                 4022
2               house                     3815
3               wedding                   2348
4               real estate other         1930
5               new construction          1881
6               commercial real estate    1315
7               buy to rent                653
8               property other             634
9               home renovation            612
dtype: int64

### Conclusion
In this section, we used lemmatization to find a small, standardized set of purpose descriptions.  Then we used a dictionary to map the original <code>purpose</code> to <code>std_purpose</code>, applying the most specific description possible. 

We also created unique id's (dictionary keys) for the <code>income_type</code> and <code>purpose_std</code> columns.

## Answer these questions

### Children and default rate
- Is there a relation between having kids and repaying a loan on time?

In [36]:
df.groupby('children')['debt'].agg(['count', 'sum', lambda x: '{:.2%} '.format(x.mean())])

,count,sum,<lambda_0>
children,,,
0,14272,1072,7.51%
1,4818,444,9.22%
2,2055,194,9.44%
3,330,27,8.18%
4,41,4,9.76%
5,9,0,0.00%


Borrowers with no children have the lowest default rate, but there is no trend toward increasing default rate as the number of children increases beyond one child.

### Conclusion
Borrowsers with no children had the lowest default rate.

### Marital status and default rate
- Is there a relation between marital status and repaying a loan on time?

In [37]:
df.groupby('family_status')['debt'].agg(['count', 'sum', lambda x: '{:.2%} '.format(x.mean())]).sort_values(by='<lambda_0>', ascending=False)

,count,sum,<lambda_0>
family_status,,,
unmarried,2813,274,9.74%
civil partnership,4177,388,9.29%
married,12380,931,7.52%
divorced,1195,85,7.11%
widow / widower,960,63,6.56%


### Conclusion
Unmarried borrwers had the highest default rate.

### Income and default rate
- Is there a relation between income level and repaying a loan on time?

In [38]:
num_buckets = 10
labels = [str(i) for i in range(1,num_buckets+1)]
#labels = None
buckets = pd.qcut(df['total_income'], num_buckets, labels)
df.groupby(buckets)['debt'].agg(['count', 'sum', lambda x: '{:.2%} '.format(x.mean())])

,count,sum,<lambda_0>
total_income,,,
1,2153,158,7.34%
2,2152,187,8.69%
3,2334,196,8.40%
4,1971,163,8.27%
5,2153,206,9.57%
6,2152,178,8.27%
7,2152,177,8.22%
8,2153,174,8.08%
9,2152,151,7.02%


### Conclusion
With the exception of the very lowest income group, the default rate decreases fairly consistently as income increases.

### Loan purpose and default rate
- How do different loan purposes affect on-time repayment of the loan?

In [39]:
df.groupby('purpose_std')['debt'].agg(['count', 'sum', lambda x: '{:.2%} '.format(x.mean())]).sort_values(by='<lambda_0>', ascending=False)

,count,sum,<lambda_0>
purpose_std,,,
car,4315,403,9.34%
education,4022,370,9.20%
buy to rent,653,52,7.96%
wedding,2348,186,7.92%
real estate other,1930,148,7.67%
new construction,1881,144,7.66%
commercial real estate,1315,99,7.53%
house,3815,262,6.87%
property other,634,42,6.62%


### Conclusion
Car and education loans have the highest default rates. Real estate-related loans have the lowest default rates.

## General conclusion
The ideal borrower has no children, is not unmarried (could be married, in a civil partership, divorced, etc.), has a high income, and is borrowing money for a home renovation. The average default rate for such borrowers is:

In [40]:
print('{:.2%}'.format(
    df[(df['children'] == 0) &
   (df['family_status'] != 'unmarried') &
   (df['total_income'] > 43000) &
   (df['purpose_std'] == 'home renovation')]['debt'].mean())
     )

3.33%


The highest-risk borrower has children, is unmarried, has a low income, and is borrowing money to buy a car.  The average default rate for these borrowers is:

In [41]:
print('{:.2%}'.format(
    df[(df['children'] > 0) &
   (df['family_status'] == 'unmarried') &
   (df['total_income'] < 30000) &
   (df['purpose_std'] == 'car')]['debt'].mean())
     )

17.44%
